In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install torch_geometric Bio

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.optim as optim

from Bio import SeqIO
from io import StringIO
from itertools import product
from tqdm import trange, tqdm

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Define sequence and identifier
sequence = "ATCGATCG"
identifier = "Sequence1"

# Create a Seq object from the sequence
seq_object = Seq(sequence)

# Create a SeqRecord object
seq_record = SeqRecord(seq_object, id=identifier)

print(seq_record)


In [ ]:
def get_kmer_dictionary(kmer_size):
    alphabet = ['A', 'C', 'G', 'T', 'N']
    kmers = [''.join(mer) for mer in product(alphabet, repeat=kmer_size)]
    kmer_dict = {kmer: i for i, kmer in enumerate(kmers)}
    return kmer_dict

In [ ]:
def get_edge_index_for_sequence(seq_record, kmer_size):
  local_dict = get_kmer_dictionary(kmer_size)
  print(local_dict)
  sources = []
  targets = []
  for i in range(0, (len(str(seq_record.seq)) - kmer_size - 1)):
    current_block = seq_record.seq[i:i+kmer_size]
    target_block = seq_record.seq[i+1:i+kmer_size+1]
    current_node = local_dict[current_block]
    target_block = local_dict[target_block]

    sources.append(current_node)
    targets.append(target_block)

  return torch.tensor([sources, targets])


# edges = get_edge_index_for_sequence(seq_record, 2)
# print(edges)

In [ ]:
%cd drive/MyDrive/10708/10708\ Project

In [ ]:

fasta_file = "sequences.fasta"

sequences = []
records = list(SeqIO.parse("sequences.fasta", "fasta"))

# for record in sequences[:5]:
#     print("ID:", record.id)
#     print("Description:", record.description)
#     print("Sequence:", record.seq)
#     print()


In [ ]:
records

In [ ]:
print(get_edge_index_for_sequence(records[0], 1).shape)

In [ ]:
#FRAGMENTED SEQUENCES NOT FULL
from Bio import SeqIO
from tqdm import tqdm
fasta_file = "sequences.fasta"

sequences = []
records = list(SeqIO.parse("sequences.fasta", "fasta"))


def get_edge_index_for_sequence(seq, kmer_size):
    local_dict = get_kmer_dictionary(kmer_size)
    # print(local_dict)
    sources = []
    targets = []
    for i in range(0, (len(seq) - kmer_size - 1)):
      current_block = seq[i:i+kmer_size]
      target_block = seq[i+1:i+kmer_size+1]
      current_node = local_dict[current_block]
      target_block = local_dict[target_block]

      sources.append(current_node)
      targets.append(target_block)

    return torch.tensor([sources, targets])
def get_subsequences(sequence, subseq_length):
    """Generate subsequences of the given length from a sequence."""
    results = []
    for i in range(len(sequence) - subseq_length + 1):
        if i + subseq_length > len(sequence):
          break
        results.append(sequence[i:i + subseq_length])
    return results

train_data = []
subseq_length = 100 # Length of each subsequence
kmer_size = 3      # Size of k-mer

for record in records:
    # Retrieve the sequence from the record
    sequence = str(record.seq)
    # For each subsequence of length 50:
    subseqs = get_subsequences(sequence, subseq_length)
    for i in tqdm(range(len(subseqs))):
        subseq = subseqs[i]
        edge_index = get_edge_index_for_sequence(subseq, kmer_size)
        train_data.append(edge_index)


In [ ]:
#non fragmented
train_data = []
for record in records:
    train_data.append(get_edge_index_for_sequence(sequence, 2))



In [ ]:
print(len(train_data), train_data[0].shape)

In [ ]:
trunc_size = min(dat.shape[1] for dat in train_data)

# for dat in train_data:
#   print(dat.shape[1])

In [ ]:
trunc_size

In [ ]:
# Truncate all the sequences to trunc_size - 5

for i in range(len(train_data)):
  train_data[i] = train_data[i][:,:trunc_size-5]


for dat in train_data:
  print(dat.shape)

In [ ]:
train_data = torch.stack(train_data)

In [ ]:
train_data

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_edges, num_nodes):
        super(GCN, self).__init__()

        self.num_nodes = num_nodes

        # Initialize GCNConv layers
        self.conv1 = GCNConv(in_channels, 1)

        self.conv_mu = GCNConv(1, out_channels)
        self.conv_logvar = GCNConv(1, out_channels)

        # Decoder for adjacency matrix
        self.decoder_linear = torch.nn.Linear(num_nodes, 16)
        self.decoder_ReLU = torch.nn.ReLU()
        self.decoder_linear2 =  torch.nn.Linear(16, num_edges)

    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, edge_index):
        print("a", x.shape, edge_index.shape)
        x = F.relu(self.conv1(x, edge_index))
        print("b", x.shape)
        mu = self.conv_mu(x, edge_index)
        print("c")
        logvar = self.conv_logvar(x, edge_index)
        print("d")
        z = self.reparameterize(mu, logvar).T

        print("e")
        # print("z shape after reparameterizing", z.shape)
        decoder1 = self.decoder_linear(z)
        print("f")
        # print("z shape after linear", z.shape)
        print("g")
        decoder2 = self.decoder_ReLU(decoder1)
        # print("z shape are relu", z.shape)
        print("h")
        decoder_edge_index = self.decoder_linear2(decoder2)
        # print("decoder_edge_index shape", decoder_edge_index.shape)
        return decoder_edge_index, mu, logvar, z

def loss_function(mse_loss_fn, recon_edge_index, true_edge_index, mu, logvar):
   # Initialize here
    BCE_adj = mse_loss_fn(recon_edge_index, true_edge_index)  # Correctly apply the loss function
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE_adj + KLD



In [ ]:
print(num_nodes)

In [ ]:
#global data
num_nodes = len(get_kmer_dictionary(2))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#in_channels, out_channels, num_edges, num_node
model = GCN(1, 2, 20, 20)
mse_loss_fn = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print(num_nodes)
# #initialize training data
# edges = torch.tensor([[0,1,2,3,4],[1,2,3,4,0]], dtype=torch.long)
# x = torch.randn(num_nodes, 1)
# adj = torch.eye(num_nodes)
# graph_data = Data(x=x, edge_index=edges)

# edges = torch.tensor([[0,1,2,3,4],[4,1,2,0,3]], dtype=torch.long)
# x = torch.randn(num_nodes, 1)
# adj = torch.eye(num_nodes)
# graph_data2 = Data(x=x, edge_index=edges)

# print(graph_data.edge_index)
# print(graph_data2.edge_index)


In [ ]:
print(model)

In [ ]:
# Training loop
losses = []
for epoch in range(200):
  curr_loss = 0
  for i in range(len(train_data)):
    edges = train_data[i]
    x = torch.randn(25, 1)
    graph_data = Data(x=x, edge_index=edges)
    print(graph_data)
    model.train()
    optimizer.zero_grad()

    recon_edges, mu, logvar, z = model(graph_data.x, graph_data.edge_index)
    # print("edges shape", edges.shape, "recon_edges shape", recon_edges.shape)
    loss = loss_function(mse_loss_fn, recon_edges, edges.float(), mu, logvar)
    curr_loss += loss.item()
    loss.backward()
    optimizer.step()
  losses.append(curr_loss/len(train_data))
  print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [ ]:
saved_losses = losses

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.plot(np.arange(1,len(losses)+ 1), saved_losses)
plt.show()

In [ ]:
torch.save(model.state_dict(), "VAE.pt")